In [4]:
import pandas as pd
import json
import re
import ast
import numpy as np
import matplotlib.pyplot as plt
from openai import OpenAI
from tqdm import tqdm
from pydantic import BaseModel

# Load the data
datas = pd.read_csv("./Data/gsm_data/gsm_train.tsv", sep="\t")


class Step(BaseModel):
    explanation: str
    output: str

class ListResponse(BaseModel):
    steps: list[Step]
    final_answer: list[str]

class FloatResponse(BaseModel):
    steps: list[Step]
    final_answer: float

class FlistResponse(BaseModel):
    steps: list[Step]
    final_answer: list[float]
    
client = OpenAI(
    base_url="https://api.chatanywhere.tech/v1"
)

def chat(says,outformat):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are an experienced optimization master."},
            {"role": "user", "content": says},
        ],
        response_format=outformat,
    )
    message = completion.choices[0].message
    if message.parsed:
        #print(message.parsed.steps)
        print(message.parsed.final_answer)
        return message.parsed.final_answer
    else:
        print(message.refusal)
        chat(says)
    
def Getvalue(ch,str):
    s = str.find(ch)
    e = str[s+1:].find(ch)
    return str[s+1:s+e+1]

meta_prompt = "Instructions that can improve the accuracy and effectiveness of the model often guide the model to gradually reason, analyze in detail, think structurally, and collaborate to solve problems."
ini_prompt = [{"instruction": "Let’s think step by step."},{"instruction": "Let’s think carefully about the problem and solve it together."},{"instruction":" Break this down"}]

def Obj(INS):
    s = 0
    for i in range(20):
        quesion = datas.iloc[20+i,0] + INS
        answer = chat(quesion,FloatResponse)
        if answer == datas.iloc[20+i,1]:
            s += 1
    return s/20
    
    
def warm_strat(num):
    pt = (
        "You need to generate {} instructions to enhance the performance of LLM."
        "This is a summary of experiences that may help you."
        "{}"
        "Common instructions include: "
        "{}"
        "Please give your recommended instructions.").format(num,json.dumps(meta_prompt),meta_prompt)
    datas = chat(pt,ListResponse)
    return datas

def candidate_sampling(history,num):
    pt = (
        "Based on the previous optimization results {}, you need to provide {} candidate instruction for the next optimization."
        "Please do not provide duplicate values."
        "This is a summary of experiences that may help you."
        "{}"
        "Please give your recommended instructions.").format(json.dumps(history),num,meta_prompt)
    datas = chat(pt,ListResponse)
    return datas

def SurrogateModel(history,samples):
    data_pred = []
    pt = ("You need to estimate the accuracy of this instruction on the gsm(A dataset of mathematical question and answer.) dataset."
        "This is a summary of experiences that may help you."
        "{}"
        "Below is the historical evaluation data"
        "{}"
        "Please guess the accuracy for these instructions as follow:"
        "{}").format(json.dumps(history),meta_prompt,samples)
    datas = chat(pt,FlistResponse)
    return datas

In [5]:
### config
numiters = 10        # number of iters for BO
numinipoint = 2      # number of ini points
numsamples = 2       # number of sampled points

arrloss = [0]
ins = warm_strat(2)
pairs = []
for i in range(numinipoint):
    s = Obj(ins[i])
    arrloss[0] = max(arrloss[0],s)
    pairs.append({"instruction": ins[i], "accuracy": s})

['1. "Follow a step-by-step approach when solving problems by clearly outlining each phase from problem definition to solution verification, ensuring nothing is overlooked."', '2. "Engage in self-dialogue by questioning the validity of assumptions and exploring alternative solutions to enhance the depth and reliability of your response."']
1080.0
7.0
5.0
62.0
110.0
400.0
400.0
8.0
1000.0
6.0
1200.0
10.0
26.1667
5250.0
36.0
15.0
5.0
9.0
15.0
476.0
1080.0
7.0
5.0
62.0
110.0
400.0
400.0
8.0
1000.0
6.0
1200.0
10.0
26.166666666666668
5250.0
36.0
15.0
5.0
9.0
15.0
476.0


In [8]:
for i in range(10):
    ins_sp = candidate_sampling(pairs,numsamples)
    s = SurrogateModel(pairs,ins_sp)
    index = s.index(max(s))
    pairs.append({"instruction": ins_sp[index], "accuracy": Obj(ins_sp[index])})

['Approach each problem with a structured methodology by defining the problem, breaking it into smaller parts, analyzing each component, applying relevant techniques, and finally verifying your solution for accuracy and validity.', 'Critically analyze each proposed solution by evaluating its strengths and weaknesses, considering alternative approaches, and testing for different scenarios to ensure robustness and comprehensiveness.']


AttributeError: 'NoneType' object has no attribute 'index'